In [3]:
# imports
import polars as pl
import httpx

In [26]:
from typing import List, Optional

from pydantic import BaseModel, Field


class PlayerRating(BaseModel):
    acceleration_diff: Optional[int] = None
    acceleration_rating: int
    age: int
    age_diff: Optional[int] = None
    agility_diff: Optional[int] = None
    agility_rating: int
    archetype: str
    awareness_diff: Optional[int] = None
    awareness_rating: int
    bCVision_diff: Optional[int] = None
    bCVision_rating: int
    blockShedding_diff: Optional[int] = None
    blockShedding_rating: int
    breakSack_diff: Optional[int] = None
    breakSack_rating: int
    breakTackle_diff: Optional[int] = None
    breakTackle_rating: int
    carrying_diff: Optional[int] = None
    carrying_rating: int
    catchInTraffic_diff: Optional[int] = None
    catchInTraffic_rating: int
    catching_diff: Optional[int] = None
    catching_rating: int
    changeOfDirection_diff: Optional[int] = None
    changeOfDirection_rating: int
    college: str
    deepRouteRunning_diff: Optional[int] = None
    deepRouteRunning_rating: int
    finesseMoves_diff: Optional[int] = None
    finesseMoves_rating: int
    firstName: str
    fullNameForSearch: str
    height: int
    height_diff: Optional[int] = None
    hitPower_diff: Optional[int] = None
    hitPower_rating: int
    impactBlocking_diff: Optional[int] = None
    impactBlocking_rating: int
    injury_diff: Optional[int] = None
    injury_rating: int
    iteration: str
    jerseyNum: int
    jerseyNum_diff: Optional[int] = None
    jumping_diff: Optional[int] = None
    jumping_rating: int
    jukeMove_diff: Optional[int] = None
    jukeMove_rating: int
    kickAccuracy_diff: Optional[int] = None
    kickAccuracy_rating: int
    kickPower_diff: Optional[int] = None
    kickPower_rating: int
    kickReturn_diff: Optional[int] = None
    kickReturn_rating: int
    lastName: str
    leadBlock_diff: Optional[int] = None
    leadBlock_rating: int
    manCoverage_diff: Optional[int] = None
    manCoverage_rating: int
    mediumRouteRunning_diff: Optional[int] = None
    mediumRouteRunning_rating: int
    overall_diff: Optional[int] = None
    overall_rating: int
    passBlock_diff: Optional[int] = None
    passBlockFinesse_diff: Optional[int] = None
    passBlockFinesse_rating: int
    passBlockPower_diff: Optional[int] = None
    passBlockPower_rating: int
    passBlock_rating: int
    playAction_diff: Optional[int] = None
    playAction_rating: int
    playRecognition_diff: Optional[int] = None
    playRecognition_rating: int
    plyrAssetname: str
    plyrBirthdate: str
    plyrHandedness: str
    plyrPortrait: int
    plyrPortrait_diff: Optional[int] = None
    position: str
    powerMoves_diff: Optional[int] = None
    powerMoves_rating: int
    press_diff: Optional[int] = None
    press_rating: int
    primaryKey: int
    primaryKey_diff: Optional[int] = None
    pursuit_diff: Optional[int] = None
    pursuit_rating: int
    release_diff: Optional[int] = None
    release_rating: int
    runBlock_diff: Optional[int] = None
    runBlockFinesse_diff: Optional[int] = None
    runBlockFinesse_rating: int
    runBlockPower_diff: Optional[int] = None
    runBlockPower_rating: int
    runBlock_rating: int
    runningStyle_rating: str
    shortRouteRunning_diff: Optional[int] = None
    shortRouteRunning_rating: int
    signingBonus: int
    signingBonus_diff: Optional[int] = None
    spectacularCatch_diff: Optional[int] = None
    spectacularCatch_rating: int
    speed_diff: Optional[int] = None
    speed_rating: int
    spinMove_diff: Optional[int] = None
    spinMove_rating: int
    stamina_diff: Optional[int] = None
    stamina_rating: int
    status: str
    stiffArm_diff: Optional[int] = None
    stiffArm_rating: int
    strength_diff: Optional[int] = None
    strength_rating: int
    tackle_diff: Optional[int] = None
    tackle_rating: int
    team: str = Field(alias="ost")
    teamId: int
    teamId_diff: Optional[int] = None
    throwAccuracyDeep_diff: Optional[int] = None
    throwAccuracyDeep_rating: int
    throwAccuracyMid_diff: Optional[int] = None
    throwAccuracyMid_rating: int
    throwAccuracyShort_diff: Optional[int] = None
    throwAccuracyShort_rating: int
    throwOnTheRun_diff: Optional[int] = None
    throwOnTheRun_rating: int
    throwPower_diff: Optional[int] = None
    throwPower_rating: int
    throwUnderPressure_diff: Optional[int] = None
    throwUnderPressure_rating: int
    totalSalary: int
    totalSalary_diff: Optional[int] = None
    toughness_diff: Optional[int] = None
    toughness_rating: int
    trucking_diff: Optional[int] = None
    trucking_rating: int
    weight: int
    weight_diff: Optional[int] = None
    yearsPro: int
    yearsPro_diff: Optional[int] = None
    zoneCoverage_diff: Optional[int] = None
    zoneCoverage_rating: int

    class Config:
        populate_by_name = True


class RatingsResponse(BaseModel):
    count: int
    docs: List[PlayerRating]


In [27]:
BASE_URL = "https://ratings-api.ea.com/v2/entities"

In [28]:
# Create a Function to Read Data from the EA Sports Madden API
def get_madden_ratings(game_version: str, iteration: str) -> list[PlayerRating]:
    url = f"{BASE_URL}/{game_version}-ratings?filter=iteration:{iteration}"
    all_ratings = []

    with httpx.Client() as client:
        response = client.get(url)
        data = response.json()
        ratings_response = RatingsResponse(**data)
        all_ratings.extend(ratings_response.docs)

        total_count = ratings_response.count
        while len(all_ratings) < total_count:
            next_url = f"{url}&limit=100&offset={len(all_ratings)}"
            response = client.get(next_url)
            data = response.json()
            ratings_response = RatingsResponse(**data)
            all_ratings.extend(ratings_response.docs)

    return all_ratings


In [5]:
# Create a function to read parquet data from the NFL Verse repo
def get_nflverse_data():
    url = 'https://github.com/nflverse/nflverse-data/releases/download/rosters/roster_2024.parquet'
    df = pl.read_parquet(url)
    return df


In [38]:
nflverse_data = get_nflverse_data()

In [21]:
# Create a Function that Gets the Headers of the dataframe and stores them in a list
def get_headers(df):
    headers_with_types = [(col, df[col].dtype) for col in df.columns]
    # Sort by data type first, then alphabetically within each data type
    sorted_headers = sorted(headers_with_types, key=lambda x: (str(x[1]), x[0]))
    sorted_headers_list = [col for col, dtype in sorted_headers]
    return sorted_headers, sorted_headers_list


In [22]:
sorted_headers, sorted_headers_list = get_headers(data)

In [23]:
sorted_headers

[('birth_date', Date),
 ('height', Float64),
 ('draft_number', Int32),
 ('entry_year', Int32),
 ('jersey_number', Int32),
 ('rookie_year', Int32),
 ('season', Int32),
 ('week', Int32),
 ('weight', Int32),
 ('years_exp', Int32),
 ('college', String),
 ('depth_chart_position', String),
 ('draft_club', String),
 ('esb_id', String),
 ('espn_id', String),
 ('fantasy_data_id', String),
 ('first_name', String),
 ('football_name', String),
 ('full_name', String),
 ('game_type', String),
 ('gsis_id', String),
 ('gsis_it_id', String),
 ('headshot_url', String),
 ('last_name', String),
 ('ngs_position', String),
 ('pff_id', String),
 ('pfr_id', String),
 ('position', String),
 ('rotowire_id', String),
 ('sleeper_id', String),
 ('smart_id', String),
 ('sportradar_id', String),
 ('status', String),
 ('status_description_abbr', String),
 ('team', String),
 ('yahoo_id', String)]

In [71]:
madden_data = get_madden_ratings("m24", "super-bowl")

In [43]:
# Count the Number
len(madden_data)

# Count the number of distinct players
len(set([player.primaryKey for player in madden_data]))

# Let's convert it to a DataFrame
madden_df = pl.DataFrame(madden_data)

In [40]:
# Let's convert the nflverse data to a DataFrame
nflverse_df = get_nflverse_data()


In [45]:
madden_df.head()

acceleration_diff,acceleration_rating,age,age_diff,agility_diff,agility_rating,archetype,awareness_diff,awareness_rating,bCVision_diff,bCVision_rating,blockShedding_diff,blockShedding_rating,breakSack_diff,breakSack_rating,breakTackle_diff,breakTackle_rating,carrying_diff,carrying_rating,catchInTraffic_diff,catchInTraffic_rating,catching_diff,catching_rating,changeOfDirection_diff,changeOfDirection_rating,college,deepRouteRunning_diff,deepRouteRunning_rating,finesseMoves_diff,finesseMoves_rating,firstName,fullNameForSearch,height,height_diff,hitPower_diff,hitPower_rating,impactBlocking_diff,…,spinMove_rating,stamina_diff,stamina_rating,status,stiffArm_diff,stiffArm_rating,strength_diff,strength_rating,tackle_diff,tackle_rating,team,teamId,teamId_diff,throwAccuracyDeep_diff,throwAccuracyDeep_rating,throwAccuracyMid_diff,throwAccuracyMid_rating,throwAccuracyShort_diff,throwAccuracyShort_rating,throwOnTheRun_diff,throwOnTheRun_rating,throwPower_diff,throwPower_rating,throwUnderPressure_diff,throwUnderPressure_rating,totalSalary,totalSalary_diff,toughness_diff,toughness_rating,trucking_diff,trucking_rating,weight,weight_diff,yearsPro,yearsPro_diff,zoneCoverage_diff,zoneCoverage_rating
i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,str,str,i64,i64,i64,i64,i64,…,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
0,71,30,0,0,77,"""QB_FieldGeneral""",0,61,0,52,0,21,0,58,0,30,0,59,0,20,0,26,0,75,"""Arkansas""",0,15,0,9,"""Brandon""","""Brandon Allen""",74,0,0,11,0,…,28,0,65,"""published""",0,31,0,56,0,24,"""49ers""",15,0,0,70,0,69,0,78,0,72,0,82,0,72,890000,0,0,75,0,29,209,0,7,0,0,18
0,90,23,0,0,90,"""WR_Playmaker""",0,62,0,79,0,26,0,31,0,76,0,70,0,77,0,81,0,90,"""Michigan""",0,70,0,10,"""Ronnie""","""Ronnie Bell""",72,0,0,25,0,…,72,0,81,"""published""",0,47,0,58,0,26,"""49ers""",15,0,0,21,0,26,0,30,0,32,0,47,0,26,3830000,0,0,81,0,54,192,0,0,0,0,18
0,92,25,0,0,89,"""DE_SmallerSpeedRusher""",0,99,0,30,0,85,0,12,0,21,0,38,0,27,0,45,0,73,"""Ohio State""",0,6,0,98,"""Nick""","""Nick Bosa""",76,0,0,84,0,…,32,0,87,"""published""",0,34,0,94,-2,91,"""49ers""",15,0,0,6,0,6,0,6,0,6,0,25,0,14,89000000,0,0,90,0,31,266,0,4,0,0,42
0,75,30,0,0,70,"""C_Agile""",0,84,0,10,0,30,0,12,0,5,0,44,0,15,0,40,0,63,"""UCLA""",0,5,0,15,"""Jake""","""Jake Brendel""",76,0,0,15,0,…,10,0,86,"""published""",0,10,0,84,0,41,"""49ers""",15,0,0,6,0,6,0,6,0,6,0,22,0,10,7850000,0,0,89,0,15,299,0,7,0,0,20
0,90,30,0,0,81,"""WR_Physical""",0,63,0,79,0,35,0,22,0,67,0,73,0,81,0,80,0,77,"""Georgia""",0,69,0,10,"""Chris""","""Chris Conley""",75,0,30,45,0,…,73,0,84,"""published""",0,66,0,69,5,40,"""49ers""",15,0,0,20,0,21,0,24,0,19,0,35,0,20,600000,0,0,74,0,57,205,0,8,0,0,25


In [66]:
madden_df = madden_df.with_columns(
    pl.col("plyrBirthdate").str.strptime(pl.Date, format="%Y-%m-%d")
)

In [41]:
nflverse_data.head()

season,team,position,depth_chart_position,jersey_number,status,full_name,first_name,last_name,birth_date,height,weight,college,gsis_id,espn_id,sportradar_id,yahoo_id,rotowire_id,pff_id,pfr_id,fantasy_data_id,sleeper_id,years_exp,headshot_url,ngs_position,week,game_type,status_description_abbr,football_name,esb_id,gsis_it_id,smart_id,entry_year,rookie_year,draft_club,draft_number
i32,str,str,str,i32,str,str,str,str,date,f64,i32,str,str,str,str,str,str,str,str,str,str,i32,str,str,i32,str,str,str,str,str,str,i32,i32,str,i32
2024,"""NYJ""","""QB""","""QB""",8,"""ACT""","""Aaron Rodgers""","""Aaron""","""Rodgers""",1983-12-02,74.0,225,"""California""","""00-0023459""","""8439""","""0ce48193-e2fa-466e-a986-33f751…","""7200""","""4307""","""2241""","""RodgAa00""","""2593""","""96""",19,"""https://static.www.nfl.com/ima…",null,1,"""REG""","""A01""","""Aaron""","""ROD339293""","""29851""","""3200524f-4433-9293-a3cf-ad7758…",2005,2005,"""GB""",24
2024,"""SEA""","""P""","""P""",9,"""RET""","""Jon Ryan""","""Jon""","""Ryan""",1981-11-26,72.0,217,"""Regina (Canada)""","""00-0023742""","""10238""","""f6b48c8b-66ac-47a3-aafc-5addce…","""8020""","""7975""","""3493""",null,"""3764""","""135""",18,"""https://static.www.nfl.com/ima…",null,1,"""REG""","""R02""","""Jon""","""RYA229481""","""30714""","""32005259-4122-9481-61e2-b493c1…",2006,2006,null,null
2024,"""ARI""","""K""","""K""",5,"""ACT""","""Matt Prater""","""Matt""","""Prater""",1984-08-10,70.0,201,"""Central Florida""","""00-0023853""","""11122""","""67f5e782-f91c-4536-9818-cf4a0e…","""8565""","""5051""","""3471""",null,"""549""","""17""",18,"""https://static.www.nfl.com/ima…",null,1,"""REG""","""A01""","""Matt""","""PRA143616""","""31446""","""32005052-4114-3616-ccef-012710…",2006,2006,null,null
2024,"""TEN""","""K""","""K""",6,"""ACT""","""Nick Folk""","""Nick""","""Folk""",1984-11-05,73.0,222,"""Arizona""","""00-0025565""","""10621""","""b37c621e-1125-4c35-bea0-fcabb1…","""8432""","""5365""",null,"""FolkNi20""","""11694""","""650""",17,"""https://static.www.nfl.com/ima…",null,1,"""REG""","""A01""","""Nick""","""FOL378021""","""32371""","""3200464f-4c37-8021-409f-36e78a…",2007,2007,"""DAL""",178
2024,"""CAR""","""LS""","""LS""",44,"""ACT""","""J.J. Jansen""","""Jeffrey""","""Jansen""",1986-01-20,74.0,235,"""Notre Dame""","""00-0026027""",null,null,null,null,null,null,null,null,16,"""https://static.www.nfl.com/ima…",null,1,"""REG""","""A01""","""J.J.""","""JAN681650""","""33557""","""32004a41-4e68-1650-bdb2-b3ecc6…",2008,2008,null,null


In [56]:
print(pl.__version__)

1.2.1


In [72]:
# Let's join the two dataframes on the fullNameForSearch (madden_df) and full_name (nflverse_df) and birth_date columns using Polars
full_data = nflverse_df.join(
    madden_df,
    left_on=["full_name", "birth_date"],
    right_on=["fullNameForSearch", "plyrBirthdate"],
    how="left"
)

In [77]:
filtered_df = full_data.filter(pl.col("team_right") == "Titans")

In [78]:
filtered_df

season,team,position,depth_chart_position,jersey_number,status,full_name,first_name,last_name,birth_date,height,weight,college,gsis_id,espn_id,sportradar_id,yahoo_id,rotowire_id,pff_id,pfr_id,fantasy_data_id,sleeper_id,years_exp,headshot_url,ngs_position,week,game_type,status_description_abbr,football_name,esb_id,gsis_it_id,smart_id,entry_year,rookie_year,draft_club,draft_number,acceleration_diff,…,spinMove_rating,stamina_diff,stamina_rating,status_right,stiffArm_diff,stiffArm_rating,strength_diff,strength_rating,tackle_diff,tackle_rating,team_right,teamId,teamId_diff,throwAccuracyDeep_diff,throwAccuracyDeep_rating,throwAccuracyMid_diff,throwAccuracyMid_rating,throwAccuracyShort_diff,throwAccuracyShort_rating,throwOnTheRun_diff,throwOnTheRun_rating,throwPower_diff,throwPower_rating,throwUnderPressure_diff,throwUnderPressure_rating,totalSalary,totalSalary_diff,toughness_diff,toughness_rating,trucking_diff,trucking_rating,weight_right,weight_diff,yearsPro,yearsPro_diff,zoneCoverage_diff,zoneCoverage_rating
i32,str,str,str,i32,str,str,str,str,date,f64,i32,str,str,str,str,str,str,str,str,str,str,i32,str,str,i32,str,str,str,str,str,str,i32,i32,str,i32,i64,…,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
2024,"""LAC""","""LS""","""LS""",47,"""ACT""","""Josh Harris""","""Josh""","""Harris""",1989-04-27,73.0,224,"""Auburn""","""00-0028845""","""15151""","""8bbd5520-52e9-4187-889f-bd4552…","""26168""","""3795""",null,null,"""14567""","""1238""",12,"""https://static.www.nfl.com/ima…",null,1,"""REG""","""A01""","""Josh""","""HAR000002""","""38785""","""32004841-5200-0002-4939-feff60…",2012,2012,null,null,0,…,45,0,75,"""published""",0,55,0,68,0,56,"""Chargers""",8,0,0,10,0,10,0,10,0,10,0,25,0,10,3490000,0,0,85,0,53,224,0,11,0,0,21
2024,"""CHI""","""WR""","""WR""",13,"""ACT""","""Keenan Allen""","""Keenan""","""Allen""",1992-04-27,74.0,211,"""California""","""00-0030279""","""15818""","""5f424505-f29f-433c-b3f2-1a143a…","""26699""","""8627""","""7857""","""AlleKe00""","""15076""","""1479""",11,"""https://static.www.nfl.com/ima…",null,1,"""REG""","""A01""","""Keenan""","""ALL532966""","""40024""","""3200414c-4c53-2966-dfa9-55ae46…",2013,2013,"""SD""",76,0,…,83,0,92,"""published""",0,78,0,72,0,34,"""Chargers""",8,0,0,34,0,33,0,36,0,31,0,54,0,36,41760000,0,0,93,0,71,211,0,10,0,0,26
2024,"""LAC""","""OL""","""C""",63,"""ACT""","""Corey Linsley""","""Corey""","""Linsley""",1991-07-27,75.0,301,"""Ohio State""","""00-0031019""",null,null,null,null,null,null,null,null,10,"""https://static.www.nfl.com/ima…",null,1,"""REG""","""A01""","""Corey""","""LIN775645""","""41390""","""32004c49-4e77-5645-c04e-13f546…",2014,2014,"""GB""",161,0,…,10,0,87,"""published""",0,10,0,92,0,29,"""Chargers""",8,0,0,6,0,6,0,6,0,6,0,26,0,16,46000000,0,0,88,0,24,301,0,9,0,0,12
2024,"""LAC""","""DL""","""DE""",52,"""ACT""","""Khalil Mack""","""Khalil""","""Mack""",1991-02-22,75.0,269,"""Buffalo""","""00-0031040""","""16710""","""33c74bf8-7621-48be-a769-e21970…","""27533""","""9251""","""8640""","""MackKh00""","""16323""","""2036""",10,"""https://static.www.nfl.com/ima…",null,1,"""REG""","""A01""","""Khalil""","""MAC325122""","""41231""","""32004d41-4332-5122-ef2d-7d4394…",2014,2014,"""OAK""",5,0,…,60,0,87,"""published""",0,51,0,87,0,86,"""Chargers""",8,0,0,6,0,6,0,6,0,6,0,32,0,12,28880000,0,0,92,0,55,269,0,9,0,0,59
2024,"""DAL""","""LB""","""ILB""",50,"""ACT""","""Eric Kendricks""","""Eric-Nathan""","""Kendricks""",1992-02-29,72.0,232,"""UCLA""","""00-0031554""","""2510863""","""b345f3db-d5aa-43ba-9f17-914c54…","""28433""","""10353""","""9478""","""KendEr00""","""16806""","""2350""",9,"""https://static.www.nfl.com/ima…",null,1,"""REG""","""A01""","""Eric""","""KEN101224""","""42388""","""32004b45-4e10-1224-7c6e-cc615b…",2015,2015,"""MIN""",45,0,…,35,0,91,"""published""",0,35,0,76,0,87,"""Chargers""",8,0,0,6,0,6,0,6,0,6,0,24,0,10,7000000,0,0,85,0,43,232,0,8,0,0,71
…,…,…,…,…,…

In [98]:
madden_df.filter(((pl.col("team").is_in(["Titans", "Broncos"])))).sort(by=["position", "overall_rating", "speed_rating", "stamina_rating", "strength_rating", "yearsPro", "totalSalary"], descending=True).select(["fullNameForSearch", "position", "team", "overall_rating", "speed_rating", "stamina_rating", "strength_rating", "yearsPro", "totalSalary"])

fullNameForSearch,position,team,overall_rating,speed_rating,stamina_rating,strength_rating,yearsPro,totalSalary
str,str,str,i64,i64,i64,i64,i64,i64
"""DeAndre Hopkins""","""WR""","""Titans""",89,89,95,78,10,8510000
"""Courtland Sutton""","""WR""","""Broncos""",84,91,88,74,5,45620000
"""Jerry Jeudy""","""WR""","""Broncos""",81,91,90,64,3,2500000
"""Tim Patrick""","""WR""","""Broncos""",78,89,88,69,6,15320000
"""Marvin Mims Jr""","""WR""","""Broncos""",76,92,84,55,0,4660000
…,…,…,…,…,…,…,…,…
"""Anthony Kendall""","""CB""","""Titans""",61,91,83,59,0,2700000
"""Lloyd Cushenberry III""","""C""","""Broncos""",77,63,88,85,3,5030000
"""Aaron Brewer""","""C""","""Titans""",73,69,84,79,3,4300000
